In [236]:
from Triangulation import *

import torch
import torch.optim as optim



from matplotlib import pyplot as plt

import torch.nn as nn
from torch.autograd import Variable

from  Neural_network import *


%matplotlib qt


In [252]:
def get_set_nb_of_points(point_coordinates):
    set_of_numbers=set()
    for index,_ in enumerate(point_coordinates):
        set_of_numbers.add(len(point_coordinates[index][0]))
    return set_of_numbers


def get_indices_nb_of_points(set_of_numbers,number_of_points,point_coordinates):
    indices=[]
    if number_of_points not in set_of_numbers:
        return "No such number of points for sample"
    else:
        for index,_ in enumerate(point_coordinates):
            if len(point_coordinates[index][0])==number_of_points:
                indices.append(index)
        return indices
    
def get_polygons_nb_of_points(indices):
    
    pass

def get_edge_lengths(polygon):
    polygon_edge_lengths=np.empty([polygon.shape[0]])
    for index,_ in enumerate(polygon):
        polygon_edge_lengths[index]=np.linalg.norm(polygon[(index+1)%(polygon.shape[0])]-polygon[index])
    return polygon_edge_lengths
    

def extract_lengths_angles(polygon_set):
    lengths=[]
    angles=[]
    target_edge_length=[]
    for polygons in polygons_reshaped:
        polygon=np.delete(polygons,16).reshape(8,2)
        lengths.append(get_edge_lengths(polygon))
        angles.append(np.array(np.multiply(pi/180,get_polygon_angles(polygon))))
        target_edge_length.append([polygons[16]])
    data=np.hstack([lengths,angles,target_edge_length])
    return data,lengths,angles
    
def extract_lengths_angles_in_triangle_form(polygon_set,nb_of_points):
    data,lengths,angles=extract_lengths_angles(polygon_set)
    data_reformed=np.empty([data.shape[0],3*nb_of_points+1])
    for polygon_index,polygon_lengths in enumerate(lengths.copy()):
        data_reformed[polygon_index][0:3*nb_of_points:3]=polygon_lengths[0:nb_of_points]
        data_reformed[polygon_index][1:3*(nb_of_points-1):3]=polygon_lengths[1:nb_of_points]
        data_reformed[polygon_index][3*nb_of_points-2]=polygon_lengths[0]
        data_reformed[polygon_index][2:3*nb_of_points:3]=angles[polygon_index]
        #including target edge length
        data_reformed[polygon_index][-1]=polygon_set[polygon_index][2*nb_of_points]
        
    return data_reformed

    

In [238]:
from torch.autograd.function import Function
from math import pow

class myLossfunction(Function):
    
    @staticmethod
    def forward(self,output,target):
        self.save_for_backward(output,target) 

                        
       # output=output.view(int(output.size()[0]/2),2)

        #target=target.view(int(target.size()[0]/2),2)
        distance=torch.nn.PairwiseDistance()
        result=distance(output,target)

        result=torch.FloatTensor(result)
        #self.save_for_backward(result)


        return  result 
    
    
    @staticmethod
    def backward(self,grad_output1):
        input1,target=self.saved_variables
        
        print(input1)
        #distance=torch.nn.PairwiseDistance()(input1.view(int(input1.size()[0]/2),2),target.view(int(target.size()[0]/2),2))
        distance=torch.nn.PairwiseDistance()(input1,target)

        grad_output1=(input1-target)/distance

        
        return grad_output1,None
    
    
class myOtherLossfunction(Function):
    
    @staticmethod
    def forward(self,output,target)->Variable:
        
        torch_sum=0
        for i in range(0,output.size()[1],2):
            euclidean_distance=(output[:,i]-target[:,i]).pow(2)+(output[:,i+1]-target[:,i+1]).pow(2)
            torch_sum+=euclidean_distance
        return  torch_sum

def my_torch_loss_function(a,b)->Variable:
    torch_sum=0
    for i in range(0,a.size()[1],2):
        euclidean_distance=torch.sqrt((a[:,i]-b[:,i]).pow(2)+(a[:,i+1]-b[:,i+1]).pow(2))
        torch_sum+=euclidean_distance
    return  torch_sum   
    
    

    
    

    


In [239]:
# Using convolution network
# O: output dimension
# I: Input dimensiion
# S: Stride
# P: padding
# w: kernel size
# O=(I-w-2*P)/S+1

# Included batch normalization with dropout layers

nb_of_points_output=1

class Conv_net(nn.Module):
    
    def __init__(self):
        super(Conv_net,self).__init__()
        self.conv1=nn.Conv1d(1,14,kernel_size=14,stride=1)
        self.conv2=nn.Conv1d(14,28,kernel_size=2,stride=1)

        self.bn1=nn.BatchNorm1d(num_features=28*9)
        self.fc1=nn.Linear(28*9,12)
        self.dp_1=nn.Dropout(p=0.5)

        
        self.bn2=nn.BatchNorm1d(num_features=12)
        self.fc2=nn.Linear(12,12)
        self.dp_2=nn.Dropout(p=0.5)

        
        self.bn3=nn.BatchNorm1d(num_features=12)
        self.fc3=nn.Linear(12,12)
        self.dp_3=nn.Dropout(p=0.5)
        
        self.bn4=nn.BatchNorm1d(num_features=12)
        self.fc4=nn.Linear(12,nb_of_points_output*2)
        
    def forward(self,x):
        x=F.relu(F.max_pool1d(self.conv1(x),kernel_size=2,stride=1))
        
        #x=F.relu(F.max_pool1d(self.conv2(x),kernel_size=2,stride=1))
        #x=self.conv2(x)

        x=F.relu(F.max_pool1d(self.conv2(x),kernel_size=2,stride=1))
        x=F.relu(self.fc1(self.bn1(x.view(-1,28*9))))
        x=F.relu(self.fc2( self.dp_1(self.bn2(x))))
        x=F.relu(self.fc3(self.dp_2(self.bn3(x))))

        x=self.fc4(self.bn4(self.dp_3(x)))

        return x

In [340]:
# Another convolution network involving the pairing of lengths and angles through convolution and then 
# fully connectiong in a  linear fashion
# Input : a matrix X where lengths take X/2 and angles X/2 and the target edge length

class alt_conv_net(nn.Module):
    
    def __init__(self,nb_of_filters,nb_of_hidden_nodes,out_dimension):
        super(alt_conv_net,self).__init__()
        
        self.nb_of_filters=nb_of_filters
        
        self.conv1=nn.Sequential(nn.Conv1d(in_channels=1,out_channels=nb_of_filters,stride=1,kernel_size=2),
                                nn.MaxPool1d(stride=1,kernel_size=2),nn.Tanh(inplace=True))
        self.conv2=nn.Sequential(nn.Conv1d(in_channels=1,out_channels=nb_of_filters,stride=1,kernel_size=2),
                                nn.MaxPool1d(stride=1,kernel_size=2),nn.Tanh(inplace=True))
        # The linear connections are fixed for 12 polygon example
   
        self.fc=nn.Sequential(  nn.BatchNorm1d(num_features=nb_of_filters*10+nb_of_filters*10+1),

                                nn.Linear(nb_of_filters*10+nb_of_filters*10+1,nb_of_hidden_nodes),
                                        nn.ReLU(inplace=True),
                               nn.BatchNorm1d(num_features=nb_of_hidden_nodes),
                               nn.Linear(nb_of_hidden_nodes,out_dimension) )                                                                        
        
        
        
        
    def forward(self,x):
        
        lenghts=x.narrow(1,0,1).narrow(2,0,12)
        angles=x.narrow(1,0,1).narrow(2,12,12)
        target_edge_length=x.narrow(1,0,1).narrow(2,24,1).resize(x.size()[0],1)

       
        conv_result1=self.conv1(lenghts)
        conv_result2=self.conv2(angles)        
        
        # reshape the convolution results
        
        conv_result1=conv_result1.view(-1,self.nb_of_filters*10)
        conv_result2=conv_result2.view(-1,self.nb_of_filters*10)
        
        concat_tensor=torch.cat([conv_result1,conv_result2,target_edge_length],1)
        output=self.fc(concat_tensor)
        return output
        

        
        

In [291]:
# Another convolutional network. The input of the network is : Length[index], Length[index+1], angle[index] e.g the triangles
# formed by connecting the neighbouring edges of the contour.
# The input is convoluted by a kernel of size 3 with a stride of 3.


class triangle_convoluting_net(nn.Module):
    
    def __init__(self,nb_of_filters,nb_of_hidden_nodes,out_dimension):
        super(triangle_convoluting_net,self).__init__()
        self.nb_of_filters=nb_of_filters
        
        self.conv=nn.Sequential(nn.Conv1d(in_channels=1,out_channels=nb_of_filters,stride=3,kernel_size=3),
                                #nn.Conv1d(in_channels=nb_of_filters,out_channels=nb_of_filters,stride=1,kernel_size=2),
                                #nn.MaxPool1d(stride=1,kernel_size=2),nn.ReLU(inplace=True),
                                nn.Conv1d(in_channels=nb_of_filters,out_channels=nb_of_filters,kernel_size=2,stride=1),
                                nn.MaxPool1d(stride=1,kernel_size=2),nn.ReLU(inplace=True))
       
        # The linear connections are fixed for 12 polygon example
   
        self.fc=nn.Sequential(  nn.BatchNorm1d(num_features= nb_of_filters*6+1),
                              
                                # 1rst layer
                                nn.Linear(nb_of_filters*6+1,nb_of_hidden_nodes),
                                        nn.ReLU(inplace=True),
                               nn.BatchNorm1d(num_features=nb_of_hidden_nodes),
                             
                              # 2nd layer
                              nn.Linear(nb_of_hidden_nodes,nb_of_hidden_nodes),
                                  nn.ReLU(inplace=True),
                               nn.BatchNorm1d(num_features=nb_of_hidden_nodes),
                                
                              # 3rd layer
                              nn.Linear(nb_of_hidden_nodes,nb_of_hidden_nodes),
                                  nn.ReLU(inplace=True),
                              nn.BatchNorm1d(num_features=nb_of_hidden_nodes),
                              
                            
                              nn.Linear(nb_of_hidden_nodes,out_dimension)
                             
                             
                             
                             )      

    def forward(self,x):
        
        triangles=x.narrow(1,0,1).narrow(2,0,3*8)
        target_edge_length=x.narrow(1,0,1).narrow(2,3*8,1).resize(x.size()[0],1)
        
        conv_result=self.conv(triangles)
        
        
        
        conv_result=conv_result.view(-1,self.nb_of_filters*6)
        #print(conv_result,target_edge_length)

        concat_tensor=torch.cat([conv_result,target_edge_length],1)
        output=self.fc(concat_tensor)
        
        return output



    

In [243]:
def save_network(filename,net):
    path=os.path.join('networks',filename)

    with open(path,'wb') as output:
        pickle.dump(net,output)
        
def load_network(filename):
    path=os.path.join('networks',filename)
        
    with open(path,'rb') as input:
        net=pickle.load(input)
        
    net.eval()
    return net

In [244]:
polygons=load_dataset('8_polygons.pkl')
polygons=[i for i in polygons for j in range(10)]
polygons_reshaped=[]
for polygon in polygons:
    polygons_reshaped.append(polygon.reshape(2,8))

polygons_reshaped=np.array(polygons_reshaped)
#polygons_reshaped=polygons_reshaped.reshape(60000,24)

In [245]:
point_coordinates=load_dataset('8_point_coordinates')
number_of_insertion_points=load_dataset('8_nb_of_points.pkl')
#centers_of_mass=load_dataset('12_centers_of_mass')

In [246]:
nb_of_points=1
set_of_points=get_set_nb_of_points(point_coordinates)        
indices=get_indices_nb_of_points(set_of_points,nb_of_points,point_coordinates)
indices=np.asarray(indices)
number_of_insertion_points=np.array(number_of_insertion_points)

In [253]:
polygons_reshaped.resize(len(point_coordinates),2*8)

polygons_reshaped=np.hstack([polygons_reshaped[indices],number_of_insertion_points[indices,1].reshape(len(indices),1) ])
#polygons_reshaped=polygons_reshaped[indices]


ValueError: cannot resize an array that references or is referenced
by another array in this way.  Use the resize function

In [248]:
point_coordinates=[ point_coordinates[i][0]for i in indices]
point_coordinates=np.array(point_coordinates)


In [249]:
#len(polygons_reshaped),len(indices)
#point_coordinates.shape
#centers_of_mass=centers_of_mass.reshape(60000,2)
point_coordinates=point_coordinates.reshape(polygons_reshaped.shape[0],1,2*nb_of_points)
#point_coordinates

In [254]:
# Extracting angles and length of edges

#data,_,_=extract_lengths_angles(polygons_reshaped)
data_triangle=extract_lengths_angles_in_triangle_form(polygons_reshaped,8)


In [255]:
nb_of_test_data=int(len(polygons_reshaped)*0.2)
nb_of_training_data=int(len(polygons_reshaped)-nb_of_test_data)


In [341]:

x_tensor=torch.from_numpy(data_triangle[:nb_of_training_data]).type(torch.FloatTensor)
#mixed_tensor=torch.cat((lengths_tensor[:nb_of_training_data],angles_tensor[:nb_of_training_data]),1)
#double_mixed_tensor=angles_tensor[:nb_of_training_data]

x_tensor_test=torch.from_numpy(data_triangle[nb_of_training_data:]).type(torch.FloatTensor)
#mixed_tensor_test=torch.cat((lengths_tensor[nb_of_training_data:],angles_tensor[nb_of_training_data:]),1)
#double_mixed_tensor_test=angles_tensor[nb_of_training_data:]




x_variable,x_variable_test=Variable(x_tensor),Variable(x_tensor_test)
#x_variable,x_variable_test=Variable(mixed_tensor),Variable(mixed_tensor_test)
#x_variable,x_variable_test=Variable(double_mixed_tensor),Variable(double_mixed_tensor_test)

y_tensor=torch.from_numpy(point_coordinates[:nb_of_training_data]).type(torch.FloatTensor)
y_tensor_test=torch.from_numpy(point_coordinates[nb_of_training_data:]).type(torch.FloatTensor)

y_variable,y_variable_test=Variable(y_tensor),Variable(y_tensor_test)



shuffle=torch.randperm(x_variable.shape[0])
x_variable = x_variable[shuffle]
y_variable=y_variable[shuffle]
print(y_variable.size(),x_variable.size())


torch.Size([76672, 1, 2]) torch.Size([76672, 25])


In [271]:
my_net=Net(x_variable.size()[1],y_variable.size()[2],nb_of_hidden_layers=3, nb_of_hidden_nodes=20,batch_normalization=True)

print("Training data length:",x_variable_test.size()[1],y_variable.size()[2])

Training data length: 17 2


In [272]:
optimizer = torch.optim.Adam(my_net.parameters(), lr=1e-4,weight_decay=0.2)
#optimizer = torch.optim.SGD(my_net.parameters(), lr=1e-5,weight_decay=.5,momentum=0.9)
#max_distance=0.6108970818704328
loss_func =torch.nn.MSELoss(size_average=False) 
#loss_func=myOtherLossfunction()


In [273]:
if  torch.cuda.is_available():
    loss_func.cuda()
        
    x_variable , y_variable=x_variable.cuda(), y_variable.cuda()
    x_variable_test,y_variable_test= Variable(x_tensor_test.cuda(),volatile=True),Variable(y_tensor_test.cuda(),volatile=True)
    #x_variable_test,y_variable_test= Variable(double_mixed_tensor_test.cuda(),volatile=True),Variable(y_tensor_test.cuda(),volatile=True)

    print("cuda activated")


cuda activated


In [274]:
batch_size=int(x_variable.size()[0]/32 )
nb_of_epochs=13000
my_net.cuda()
#my_net.cpu()

# Train the network #
my_net.train()
for t in range(nb_of_epochs):
    sum_loss1=0
    sum_loss2=0
    for b in range(0,x_variable.size(0),batch_size):
        out = my_net(x_variable.narrow(0,b,batch_size))                 # input x and predict based on x
        loss2= loss_func(out, y_variable.narrow(0,b,batch_size))     # must be (1. nn output, 2. target), the target label is NOT one-hotted
        #loss=loss_func.apply(out, y_variable.narrow(0,b,batch_size).resize(batch_size,2))
        
        #loss1=my_torch_loss_function(out, y_variable.narrow(0,b,batch_size).resize(batch_size,nb_of_points*2)).sum()
        #sum_loss1+=loss1.data[0]
        sum_loss2+=loss2.data[0]

        
        optimizer.zero_grad()   # clear gradients for next train
        loss2.backward()         # backpropagation, compute gradients
        #print(t,loss1.data[0],loss2.data[0])
        optimizer.step()        # apply gradients
    if t%10==0: 
        my_net.eval()
        test_loss=loss_func(my_net(x_variable_test),y_variable_test).data[0]
        #my_net.train()
        print("Epoch:",t,"Training Loss:",sum_loss2/(x_variable.size(0)),"Test Loss:",test_loss/x_variable_test.size(0))
       # print("Epoch:",t,"Training Loss:",sum_loss1/(x_variable.size(0)),sum_loss2/(x_variable.size(0)))
        my_net.train()


Epoch: 0 Training Loss: 0.3912437260648444 Test Loss: 0.36413092764470734
Epoch: 10 Training Loss: 0.14577855331471845 Test Loss: 0.1441662618035267
Epoch: 20 Training Loss: 0.08916065529710264 Test Loss: 0.09028867489109453
Epoch: 30 Training Loss: 0.0730989829128692 Test Loss: 0.07417344489758321
Epoch: 40 Training Loss: 0.0658567385601878 Test Loss: 0.06713007447715594
Epoch: 50 Training Loss: 0.05990956561991289 Test Loss: 0.06096041620474228
Epoch: 60 Training Loss: 0.0554591233224821 Test Loss: 0.0566039746114129
Epoch: 70 Training Loss: 0.052859940990581736 Test Loss: 0.05433836922621687
Epoch: 80 Training Loss: 0.05126572988666159 Test Loss: 0.05281165286973243
Epoch: 90 Training Loss: 0.050030097440009524 Test Loss: 0.05167440181980547
Epoch: 100 Training Loss: 0.04893287583464175 Test Loss: 0.05088513522394909
Epoch: 110 Training Loss: 0.047986153967193454 Test Loss: 0.05023567186970145
Epoch: 120 Training Loss: 0.047063930205789355 Test Loss: 0.0497162616710631
Epoch: 130 Tr

Epoch: 1060 Training Loss: 0.037537887418807446 Test Loss: 0.04533652749801916
Epoch: 1070 Training Loss: 0.03752943231585826 Test Loss: 0.04532991387012208
Epoch: 1080 Training Loss: 0.03752135991652143 Test Loss: 0.045321641262863234
Epoch: 1090 Training Loss: 0.03751339856690676 Test Loss: 0.045304159887246974
Epoch: 1100 Training Loss: 0.03750600788947537 Test Loss: 0.0452897226273118
Epoch: 1110 Training Loss: 0.037497997781470144 Test Loss: 0.04527174451315343
Epoch: 1120 Training Loss: 0.03748998269811894 Test Loss: 0.045260615659277505
Epoch: 1130 Training Loss: 0.03748232106334578 Test Loss: 0.04524270759799046
Epoch: 1140 Training Loss: 0.03747535368834991 Test Loss: 0.04523798539761908
Epoch: 1150 Training Loss: 0.03746794211446542 Test Loss: 0.0452296682112603
Epoch: 1160 Training Loss: 0.03746090100484221 Test Loss: 0.04522411492909732
Epoch: 1170 Training Loss: 0.03745386875133483 Test Loss: 0.04520877653449923
Epoch: 1180 Training Loss: 0.03744679211774135 Test Loss: 0.0

Epoch: 2110 Training Loss: 0.0364710104485386 Test Loss: 0.04513177887625209
Epoch: 2120 Training Loss: 0.03646510073259 Test Loss: 0.045130530661453984
Epoch: 2130 Training Loss: 0.036459235695248254 Test Loss: 0.04512614917277494
Epoch: 2140 Training Loss: 0.03645301959749454 Test Loss: 0.04511194436100369


KeyboardInterrupt: 

In [286]:
# Visualization of error

sample_index=264
sample_contour=np.delete(polygons_reshaped[sample_index],16)
#sample_contour=polygons_reshaped[sample_index]
sample_contour=sample_contour.reshape(8,2)
_,inserted_point_coordinates=get_extrapoints_target_length(sample_contour,.8)
inserted_point_coordinates=np.array(inserted_point_coordinates)

my_net.eval()
predictions=my_net(x_variable_test).cpu()
predicted_inserted_points=predictions.data.numpy()[sample_index]
predicted_inserted_points=predicted_inserted_points.reshape(nb_of_points,2)


plt.scatter(predicted_inserted_points[:,0],predicted_inserted_points[:,1],label='Predicted points')
plt.scatter(inserted_point_coordinates[:,0],inserted_point_coordinates[:,1],label='Original points')
#plt.scatter(polygons_reshaped[3112][0::2].sum()/12,polygons_reshaped[312][1::2].sum()/12,label='Original points')

plot_contour(sample_contour)
plt.legend()


#original_points=point_coordinates.reshape(218722,2)
#plt.scatter(original_points[:,0],original_points[:,1],label='Original points')


9 0.2569781749782649 0.05046506055488852 0


RuntimeError: running_mean should contain 1 elements not 17

In [226]:
save_network('8_point_regression_FNN.pkl',my_net)


In [228]:
FNN_net=load_network('8_point_regression_FNN.pkl')

In [285]:
sample_index=2641
sample_contour=np.delete(polygons_reshaped[sample_index],24)
#sample_contour=polygons_reshaped[sample_index]
sample_contour=sample_contour.reshape(12,2)
_,inserted_point_coordinates=get_extrapoints_target_length(sample_contour,8)
inserted_point_coordinates=np.array(inserted_point_coordinates)

predictions=FNN_net(x_variable_test).cpu()
predicted_inserted_points=predictions.data.numpy()[sample_index]
predicted_inserted_points=predicted_inserted_points.reshape(nb_of_points,2)


plt.scatter(predicted_inserted_points[:,0],predicted_inserted_points[:,1],label='Predicted points')
plt.scatter(inserted_point_coordinates[:,0],inserted_point_coordinates[:,1],label='Original points')
#plt.scatter(polygons_reshaped[3112][0::2].sum()/12,polygons_reshaped[312][1::2].sum()/12,label='Original points')

plot_contour(sample_contour)
plt.legend()

IndexError: index 24 is out of bounds for axis 0 with size 17

In [35]:

torch.cuda.empty_cache()

distances=[np.linalg.norm(i-j) for i in centers_of_mass for j in predicted_inserted_points]
distances=np.array(distances)
100*distances.sum()/(x_variable.size(0) *max_distance)

In [277]:
x_variable.size()

torch.Size([76672, 25])

In [342]:
x_variable,x_variable_test=x_variable.resize(x_variable.size()[0],1,x_variable.size()[1]),Variable(x_tensor_test.view(x_variable_test.size()[0],1,x_variable_test.size()[1]),volatile=True)

In [343]:
y_variable.size()
    
        

torch.Size([76672, 1, 2])

In [344]:
#torch.cuda.empty_cache()
my_conv_net=triangle_convoluting_net(nb_of_filters=3,out_dimension=2*nb_of_points,nb_of_hidden_nodes=20)
print(my_conv_net)

triangle_convoluting_net(
  (conv): Sequential(
    (0): Conv1d(1, 3, kernel_size=(3,), stride=(3,))
    (1): Conv1d(3, 3, kernel_size=(2,), stride=(1,))
    (2): MaxPool1d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
    (3): ReLU(inplace)
  )
  (fc): Sequential(
    (0): BatchNorm1d(19, eps=1e-05, momentum=0.1, affine=True)
    (1): Linear(in_features=19, out_features=20, bias=True)
    (2): ReLU(inplace)
    (3): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True)
    (4): Linear(in_features=20, out_features=20, bias=True)
    (5): ReLU(inplace)
    (6): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True)
    (7): Linear(in_features=20, out_features=20, bias=True)
    (8): ReLU(inplace)
    (9): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True)
    (10): Linear(in_features=20, out_features=2, bias=True)
  )
)


In [345]:
optimizer = torch.optim.Adam(my_conv_net.parameters(), lr=1e-4,weight_decay=0)

#optimizer = torch.optim.SGD(my_net.parameters(), lr=1e-5,weight_decay=.5,momentum=0.9)

loss_func = torch.nn.MSELoss(size_average=False) 
#loss_func=torch.nn.SmoothL1Loss()
x_variable.size()[0],y_variable.size()[0]

(76672, 76672)

In [346]:
if  torch.cuda.is_available():
    loss_func.cuda()
    my_conv_net.cuda()
    x_variable , y_variable,x_variable_test,y_variable_test= x_variable.cuda(), y_variable.cuda(),x_variable_test.cuda(),y_variable_test.cuda()
    print("cuda activated")


cuda activated


In [347]:

batch_size=int(x_variable.size()[0]/32)
nb_of_epochs=13000
# Train the network #
my_conv_net.train()
for t in range(nb_of_epochs):
    sum_loss=0
    for b in range(0,x_variable.size(0),batch_size):
        out = my_conv_net(x_variable.narrow(0,b,batch_size))                 # input x and predict based on x
        loss = loss_func(out, y_variable.narrow(0,b,batch_size))     # must be (1. nn output, 2. target), the target label is NOT one-hotted
        #loss=loss_func.apply(out, y_variable.narrow(0,b,batch_size))
        #loss=torch.sqrt((out[0]-y_variable.narrow(0,b,batch_size)[0]).pow(2)+(out[1]-y_variable.narrow(0,b,batch_size)[1]).pow(2)) 
        sum_loss+=loss.data[0]
        optimizer.zero_grad()   # clear gradients for next train
        loss.backward()         # backpropagation, compute gradients
        #print(t,loss.data[0])
        optimizer.step()        # apply gradients
    if t%10==0:
        my_conv_net.eval()
        test_loss=loss_func(my_conv_net(x_variable_test),y_variable_test).data[0]
        my_conv_net.train()
        print("Epoch:",t,"Training Loss:",sum_loss/x_variable.size(0),"Test Loss:",test_loss/x_variable_test.size(0))

Epoch: 0 Training Loss: 0.342637263474759 Test Loss: 0.10341109337910188
Epoch: 10 Training Loss: 0.10393777753355507 Test Loss: 0.10251571578852123
Epoch: 20 Training Loss: 0.08156946007914853 Test Loss: 0.08168588934438256
Epoch: 30 Training Loss: 0.07429209734641253 Test Loss: 0.07521713715363823
Epoch: 40 Training Loss: 0.07000375689568623 Test Loss: 0.07117099793805105
Epoch: 50 Training Loss: 0.0670653603510785 Test Loss: 0.06835124886692664
Epoch: 60 Training Loss: 0.06479949923309937 Test Loss: 0.06601516392473784
Epoch: 70 Training Loss: 0.06286543120127887 Test Loss: 0.064062408294423
Epoch: 80 Training Loss: 0.06079361116348801 Test Loss: 0.06210059673837907
Epoch: 90 Training Loss: 0.058530415437854394 Test Loss: 0.059982236120259026
Epoch: 100 Training Loss: 0.0572348019117505 Test Loss: 0.05901767176658363
Epoch: 110 Training Loss: 0.05625218769942779 Test Loss: 0.05832276638839798
Epoch: 120 Training Loss: 0.05538744828935856 Test Loss: 0.057691723755086605
Epoch: 130 Tr

Epoch: 1060 Training Loss: 0.04591508530615168 Test Loss: 0.05588955632434266
Epoch: 1070 Training Loss: 0.04590160769094808 Test Loss: 0.05588830810954455
Epoch: 1080 Training Loss: 0.04588915469849448 Test Loss: 0.055907044068401766
Epoch: 1090 Training Loss: 0.045874359014635296 Test Loss: 0.0559322121545349
Epoch: 1100 Training Loss: 0.0458596276122461 Test Loss: 0.055958367349309396
Epoch: 1110 Training Loss: 0.0458460172548119 Test Loss: 0.055986407603166734
Epoch: 1120 Training Loss: 0.045832733280073625 Test Loss: 0.05601036568515886
Epoch: 1130 Training Loss: 0.04581974543792776 Test Loss: 0.05603387160770881
Epoch: 1140 Training Loss: 0.04580683272350611 Test Loss: 0.056046003491333214
Epoch: 1150 Training Loss: 0.04579445077899302 Test Loss: 0.05605982301231219
Epoch: 1160 Training Loss: 0.04578086818398936 Test Loss: 0.05608215714137821
Epoch: 1170 Training Loss: 0.04576584433076378 Test Loss: 0.056096607138199084
Epoch: 1180 Training Loss: 0.04575339850280838 Test Loss: 0.

Epoch: 2110 Training Loss: 0.04504131763327699 Test Loss: 0.05672410254884443
Epoch: 2120 Training Loss: 0.045036451446393094 Test Loss: 0.05673495437545649
Epoch: 2130 Training Loss: 0.04503242828213114 Test Loss: 0.056713632828803213
Epoch: 2140 Training Loss: 0.045028705131231445 Test Loss: 0.05671690620842681
Epoch: 2150 Training Loss: 0.045023550971322546 Test Loss: 0.05673179562780415
Epoch: 2160 Training Loss: 0.04501946820241581 Test Loss: 0.05673757180546679
Epoch: 2170 Training Loss: 0.045014985216678884 Test Loss: 0.05674377466880021
Epoch: 2180 Training Loss: 0.04501157053722961 Test Loss: 0.05673077667694856
Epoch: 2190 Training Loss: 0.045006669821444655 Test Loss: 0.056732120418389376
Epoch: 2200 Training Loss: 0.045002396397280174 Test Loss: 0.056739889918663265
Epoch: 2210 Training Loss: 0.04499756264965204 Test Loss: 0.05674002365596306
Epoch: 2220 Training Loss: 0.04499362734999999 Test Loss: 0.05673346415983019
Epoch: 2230 Training Loss: 0.044989590055755466 Test Lo

Epoch: 3160 Training Loss: 0.04463453862026259 Test Loss: 0.05711834100331607
Epoch: 3170 Training Loss: 0.04462819594970729 Test Loss: 0.05713424300510617
Epoch: 3180 Training Loss: 0.04462599137390595 Test Loss: 0.057129199198370985
Epoch: 3190 Training Loss: 0.04462079810777769 Test Loss: 0.057134032846492204
Epoch: 3200 Training Loss: 0.044617606423335 Test Loss: 0.05714413956529111
Epoch: 3210 Training Loss: 0.044613125527243384 Test Loss: 0.057142579296793485
Epoch: 3220 Training Loss: 0.04460910305952986 Test Loss: 0.057145922729288395
Epoch: 3230 Training Loss: 0.044606631009128934 Test Loss: 0.057139949129897484
Epoch: 3240 Training Loss: 0.0446025671465369 Test Loss: 0.05714383388003443
Epoch: 3250 Training Loss: 0.044601255048296484 Test Loss: 0.05714026755203986
Epoch: 3260 Training Loss: 0.044597637872266054 Test Loss: 0.057126218767118375
Epoch: 3270 Training Loss: 0.04459518164346533 Test Loss: 0.0571303964656263
Epoch: 3280 Training Loss: 0.044589703688040404 Test Loss:

Epoch: 4210 Training Loss: 0.044421115084761174 Test Loss: 0.05699379336654841
Epoch: 4220 Training Loss: 0.04441812709098268 Test Loss: 0.05699072377709594
Epoch: 4230 Training Loss: 0.044416151381096176 Test Loss: 0.056986845395401844
Epoch: 4240 Training Loss: 0.04441403397335632 Test Loss: 0.05698538065354693
Epoch: 4250 Training Loss: 0.04441221210116735 Test Loss: 0.05698191622063791
Epoch: 4260 Training Loss: 0.044409753981735155 Test Loss: 0.0569743059314352
Epoch: 4270 Training Loss: 0.04440809828610173 Test Loss: 0.05697692336144551
Epoch: 4280 Training Loss: 0.044404765799367966 Test Loss: 0.05695720029394694
Epoch: 4290 Training Loss: 0.04440226618555034 Test Loss: 0.0569543217577799
Epoch: 4300 Training Loss: 0.04440037416098313 Test Loss: 0.056962244100682125
Epoch: 4310 Training Loss: 0.044399543676233055 Test Loss: 0.056955378919292574
Epoch: 4320 Training Loss: 0.04439740726267157 Test Loss: 0.05694371830043888
Epoch: 4330 Training Loss: 0.04439540199922998 Test Loss: 

...

Epoch: 8410 Training Loss: 0.04386105222972685 Test Loss: 0.05651890495185661
Epoch: 8420 Training Loss: 0.043858906960049536 Test Loss: 0.05652186627778068
Epoch: 8430 Training Loss: 0.04385595050996453 Test Loss: 0.056516191995203593
Epoch: 8440 Training Loss: 0.0438554587467684 Test Loss: 0.056515141202133766
Epoch: 8450 Training Loss: 0.04385355866214071 Test Loss: 0.056506906805532005
Epoch: 8460 Training Loss: 0.043852497918378926 Test Loss: 0.05651151118971071
Epoch: 8470 Training Loss: 0.04385202814621201 Test Loss: 0.056511364715525224
Epoch: 8480 Training Loss: 0.04385032558281951 Test Loss: 0.056498545040073296
Epoch: 8490 Training Loss: 0.043849744064382 Test Loss: 0.056511944243824344
Epoch: 8500 Training Loss: 0.043848626999703794 Test Loss: 0.05650668391003234
Epoch: 8510 Training Loss: 0.04384570816323435 Test Loss: 0.05651274666762312
Epoch: 8520 Training Loss: 0.04384518286620635 Test Loss: 0.0565091739711857
Epoch: 8530 Training Loss: 0.043842552002761916 Test Loss: 

Epoch: 9460 Training Loss: 0.04378266376326598 Test Loss: 0.0565583319815252
Epoch: 9470 Training Loss: 0.04378277272334282 Test Loss: 0.0565584529819393
Epoch: 9480 Training Loss: 0.04378246922723837 Test Loss: 0.056565553795714214
Epoch: 9490 Training Loss: 0.04378226115826971 Test Loss: 0.05657773662688139
Epoch: 9500 Training Loss: 0.043781082498808334 Test Loss: 0.05656675106296953
Epoch: 9510 Training Loss: 0.043780528643294445 Test Loss: 0.05656771906628234
Epoch: 9520 Training Loss: 0.043780063149925066 Test Loss: 0.05655597565767164
Epoch: 9530 Training Loss: 0.043779001212080254 Test Loss: 0.0565659550076136
Epoch: 9540 Training Loss: 0.043779089275704 Test Loss: 0.05656641990394146
Epoch: 9550 Training Loss: 0.04377855402798406 Test Loss: 0.05657127902583407
Epoch: 9560 Training Loss: 0.04377907862846362 Test Loss: 0.0565583319815252
Epoch: 9570 Training Loss: 0.04377740034475947 Test Loss: 0.05657441229971502
Epoch: 9580 Training Loss: 0.043776572248175466 Test Loss: 0.0565

Epoch: 10500 Training Loss: 0.04374048427269733 Test Loss: 0.05666424555452121
Epoch: 10510 Training Loss: 0.04374041103560459 Test Loss: 0.05667692512423049
Epoch: 10520 Training Loss: 0.043740736124710564 Test Loss: 0.05668119198093828
Epoch: 10530 Training Loss: 0.043739650703232755 Test Loss: 0.0566851404155037
Epoch: 10540 Training Loss: 0.043739257750407684 Test Loss: 0.056691521595236855
Epoch: 10550 Training Loss: 0.04373825710882528 Test Loss: 0.05668642684095889
Epoch: 10560 Training Loss: 0.043739824740834946 Test Loss: 0.05668361835766316
Epoch: 10570 Training Loss: 0.04373910331566863 Test Loss: 0.05668561168027442
Epoch: 10580 Training Loss: 0.043739323225960705 Test Loss: 0.0566919037018077
Epoch: 10590 Training Loss: 0.04373871523868261 Test Loss: 0.056683911306034146
Epoch: 10600 Training Loss: 0.04373710949552278 Test Loss: 0.056682019878508454
Epoch: 10610 Training Loss: 0.04373806098068696 Test Loss: 0.056682382879750755
Epoch: 10620 Training Loss: 0.043737080339995

Epoch: 11540 Training Loss: 0.04369611951066974 Test Loss: 0.05683037912308274
Epoch: 11550 Training Loss: 0.04369629374728577 Test Loss: 0.05683195849690891
Epoch: 11560 Training Loss: 0.04369657336172954 Test Loss: 0.0568312324944243
Epoch: 11570 Training Loss: 0.043696779739080566 Test Loss: 0.0568454850695169
Epoch: 11580 Training Loss: 0.04369521479375772 Test Loss: 0.0568477394982849
Epoch: 11590 Training Loss: 0.04369518852393098 Test Loss: 0.056843370746491546
Epoch: 11600 Training Loss: 0.043694782237178695 Test Loss: 0.0568505034024807
Epoch: 11610 Training Loss: 0.04369480164102799 Test Loss: 0.056851560563993375
Epoch: 11620 Training Loss: 0.043694832388666116 Test Loss: 0.05684808976264151
Epoch: 11630 Training Loss: 0.04369533380403343 Test Loss: 0.05686273081274781
Epoch: 11640 Training Loss: 0.043693807765916116 Test Loss: 0.056859533854438385
Epoch: 11650 Training Loss: 0.04369350486685318 Test Loss: 0.056861565387706724
Epoch: 11660 Training Loss: 0.04369285837039725 

Epoch: 12580 Training Loss: 0.04367350039378629 Test Loss: 0.056910908082888796
Epoch: 12590 Training Loss: 0.04367331531091604 Test Loss: 0.056912538404257744
Epoch: 12600 Training Loss: 0.04367311012764805 Test Loss: 0.056915741731010015
Epoch: 12610 Training Loss: 0.04367339382187552 Test Loss: 0.056919078795062084
Epoch: 12620 Training Loss: 0.04367242352990356 Test Loss: 0.056916505944151714
Epoch: 12630 Training Loss: 0.04367225008934288 Test Loss: 0.05692296354519903
Epoch: 12640 Training Loss: 0.04367149234415096 Test Loss: 0.05692686103222167
Epoch: 12650 Training Loss: 0.043672404126054255 Test Loss: 0.056922065594757536
Epoch: 12660 Training Loss: 0.043672114461411615 Test Loss: 0.05692144785580133
Epoch: 12670 Training Loss: 0.04367225824891028 Test Loss: 0.05691299056369992
Epoch: 12680 Training Loss: 0.043672755385480065 Test Loss: 0.056919792060661
Epoch: 12690 Training Loss: 0.04367138517519867 Test Loss: 0.056924351865739777
Epoch: 12700 Training Loss: 0.04367006441985

In [298]:
x_variable=x_variable.cpu()
my_conv_net=my_conv_net.cpu()
x_variable_test=x_variable_test.cpu()


In [299]:
#contour1=np.delete(polygons_reshaped[0],16)
#contour1=contour1.reshape(8,2)
#plot_contour(contour1)
save_network('8_point_regression_CNN.pkl',my_conv_net)

In [310]:
sample_index=4

contour1=np.delete(polygons_reshaped[sample_index],16)
contour1=contour1.reshape(8,2)
nb_of_inserted_points,inserted_point_coordinates=get_extrapoints_target_length(contour1,.8
                                                                              )
inserted_point_coordinates=np.array(inserted_point_coordinates)
my_conv_net.eval()
predictions=my_conv_net(x_variable_test)
predicted_inserted_points=predictions.data.numpy()[sample_index]
predicted_inserted_points=predicted_inserted_points.reshape(1,2)
plt.scatter(predicted_inserted_points[:,0],predicted_inserted_points[:,1],label='Predicted points')
plt.scatter(inserted_point_coordinates[:,0],inserted_point_coordinates[:,1],label='Original points')

plot_contour(contour1)
plt.legend()

9 -0.299865726930685 0.2005578975693147 -0


In [311]:
contour=polygons_reshaped[1]
contour

array([ 1.04086411, -0.1030227 ,  0.42048059,  0.45060827, -0.07548485,
        0.9862254 , -0.45504499,  0.62012031, -1.10505747,  0.2925037 ,
       -0.62290614, -1.0617667 , -0.05984104, -0.56100069,  0.85698979,
       -0.6236676 ,  0.4       ])

In [ ]:
d_tensor=torch.FloatTensor([[2.2,1.2],[3,3]])
a_tensor=torch.FloatTensor([[1,2],[3,2]])


In [ ]:
d_variable=Variable(d_tensor)

a_variable=Variable(a_tensor)




In [ ]:
d_variable.requires_grad=False
a_variable.requires_grad=True

In [ ]:
#fun=torch.sqrt((a_variable[0]-d_variable[0]).pow(2)+(a_variable[1]-d_variable[1]).pow(2))
#print(fun)

    
loss_result=my_torch_loss_function(a_variable,d_variable) 

loss_result.sum()

In [ ]:
loss_result.sum().backward()


In [ ]:
a_variable.grad

In [ ]:
another_loss_func=torch.nn.MSELoss(size_average=False)
another_loss_result=another_loss_func(out,y_variable.view(0,b,batch_size))

another_loss_result


In [ ]:
myloss=myLossfunction().apply(out,y_variable.narrow(0,b,batch_size).resize(batch_size,2))
myloss.sum().backward()

In [ ]:
# Flush all variables in GPU
del my_conv_net,my_net,x_variable_test,x_variable,y_variable,y_variable_test,loss_func
torch.cuda.empty_cache()


In [ ]:
'''
Should I order the interior points ?

For each interior point calculate distance with points of the polygon

find which ti which point it is closer and sort it according to the index.

If multiple points are closer to the same point of the contour then take into

account the distance to the point.

'''


In [232]:
# Comparing convoutional network with FNN




CNN_net=load_network('12_point_regression_CNN.pkl')
FNN_net=load_network('12_point_regression_FNN.pkl')
FNN_net=FFN_net.cuda()


In [235]:
sample_index=63

contour1=np.delete(polygons_reshaped[sample_index],24)
contour1=contour1.reshape(12,2)
nb_of_inserted_points,inserted_point_coordinates=get_extrapoints_target_length(contour1,.3                                                                            )
inserted_point_coordinates=np.array(inserted_point_coordinates)


predictions=CNN_net(x_variable_test)
conv_predicted_inserted_points=predictions.data.numpy()[sample_index]
conv_predicted_inserted_points=predicted_inserted_points.reshape(1,2)
print("convolution prediction", conv_predicted_inserted_points[0])

x_variable_test_resized=x_variable_test.resize(x_variable_test.size()[0],x_variable_test.size()[2]).cpu()
predictions=FNN_net(x_variable_test_resized.cuda()).cpu()
fnn_predicted_inserted_points=predictions.data.numpy()[sample_index]
fnn_predicted_inserted_points=predicted_inserted_points.reshape(1,2)
print("feedforward prediction",fnn_predicted_inserted_points[0])



plt.scatter(fnn_predicted_inserted_points[:,0],fnn_predicted_inserted_points[:,1],label='fnn predicted points')
plt.scatter(conv_predicted_inserted_points[:,0],conv_predicted_inserted_points[:,1],label='cnn predicted points')
plt.scatter(inserted_point_coordinates[:,0],inserted_point_coordinates[:,1],label='Original points')

plot_contour(contour1)
plt.legend()

13 -0.3493427200727041 -0.2353175009276941 0
convolution prediction [-0.20658945 -0.05526882]
feedforward prediction [-0.20658945 -0.05526882]


In [100]:
x_variable_test_resized=x_variable_test.resize(x_variable_test.size()[0],x_variable_test.size()[2])

In [109]:
x_variable_test

Variable containing:
(  0  ,.,.) = 
  0.8113  0.3627  2.3743  ...   0.8113  3.0093  0.8000

(  1  ,.,.) = 
  0.8113  0.3627  2.3743  ...   0.8113  3.0093  0.9000

(  2  ,.,.) = 
  0.8113  0.3627  2.3743  ...   0.8113  3.0093  1.0000
 ...  

(43741,.,.) = 
  0.6288  0.3673  2.6870  ...   0.6288  2.5052  0.8000

(43742,.,.) = 
  0.6288  0.3673  2.6870  ...   0.6288  2.5052  0.9000

(43743,.,.) = 
  0.6288  0.3673  2.6870  ...   0.6288  2.5052  1.0000
[torch.FloatTensor of size 43744x1x37]

In [122]:
FNN_net.cpu()

Net(
  (bn_input): BatchNorm1d(37, eps=1e-05, momentum=0.5, affine=True)
  (fc0): Linear(in_features=37, out_features=20, bias=True)
  (bn0): BatchNorm1d(20, eps=1e-05, momentum=0.5, affine=True)
  (predict): Linear(in_features=20, out_features=2, bias=True)
  (fc1): Linear(in_features=20, out_features=20, bias=True)
  (bn1): BatchNorm1d(20, eps=1e-05, momentum=0.5, affine=True)
  (fc2): Linear(in_features=20, out_features=20, bias=True)
  (bn2): BatchNorm1d(20, eps=1e-05, momentum=0.5, affine=True)
)

In [118]:
x_tensor_test


 0.8113  0.3627  2.3743  ...   0.8113  3.0093  0.8000
 0.8113  0.3627  2.3743  ...   0.8113  3.0093  0.9000
 0.8113  0.3627  2.3743  ...   0.8113  3.0093  1.0000
          ...             ⋱             ...          
 0.6288  0.3673  2.6870  ...   0.6288  2.5052  0.8000
 0.6288  0.3673  2.6870  ...   0.6288  2.5052  0.9000
 0.6288  0.3673  2.6870  ...   0.6288  2.5052  1.0000
[torch.FloatTensor of size 43744x37]

In [336]:
y_variable_test=y_variable_test.cpu().resize(y_variable_test.size()[0],y_variable_test.size()[2])

In [338]:

plot_contour(contour1)
plt.scatter(predictions.data.numpy()[:,0],predictions.data.numpy()[:,1])
plt.scatter(y_variable_test.data.numpy()[:,0],y_variable_test.data.numpy()[:,1])



In [339]:
y_variable

Variable containing:
 0.2443 -0.2558
-0.1168  0.3184
 0.0416  0.2039
       ⋮        
-0.1278 -0.3300
 0.4185  0.0228
-0.3816  0.2338
[torch.FloatTensor of size 76672x2]